In [7]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from time import sleep
import datetime
import pandas as pd
import json

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException, UnexpectedAlertPresentException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

options = Options()
# options.headless = True
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.set_window_position(0, 0)
driver.set_window_size(1400, 768)

In [12]:
sa = gspread.service_account(filename="./credentials/jp-macdonnel-b7aa48547774.json")
sh = sa.open("green-lights")
sheet = sh.worksheet("2022-11-28")

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

class GreenLight:

  def __init__(self, sheet):
    self.sheet = sheet
    self.df = pd.DataFrame(sheet.get_all_records())
    self.filled_df = self.df[['Customer Name', 'First Name', 'Last Name', 'Job Title', 'Work Phone', 'Email', 'Address Line 1', 'City', 'State', 'Zip Code']]
    self.username = None
    self.password = None
    self.logged_in = False
    self.attempts = 0
    self.input_names = {
      "customer_name": "s_1_1_86_0",
      "first_name": "s_1_1_78_0",
      "last_name": "s_1_1_77_0",
      "job_title": "s_1_1_80_0",
      "email": "s_1_1_99_0",
      "work_phone": "s_1_1_82_0",
      "address_line_1": "s_1_1_94_0",
      "city": "s_1_1_95_0",
      "state": "s_1_1_96_0",
      "zip_code": "s_1_1_97_0",
      "acct_team_owner": "s_1_1_90_0",
      "franchise_name": "s_1_1_61_0",
      "primary_opportunity": "s_1_1_71_0",
      "expected_quantity": "s_1_1_72_0"
    }

  def attempt(self, callback):
    if self.attempts >= 50:
      self.attempts = 0
      raise Exception("attempt limit exceeded")
    print(f"attempting {callback.__name__} for the {self.attempts + 1} time")
    sleep(1)
    self.attempts += 1
    return callback()

  def get_credentials(self):
    file = open("./credentials/unishipper-credentials.json")
    data = json.load(file)
    self.username = data['username']
    self.password = data['password']

  def login(self):
    if not self.username or not self.password:
      self.get_credentials()
    driver.get("http://uone.unishippers.com")
    sleep(3)
    try:
      input_username = driver.find_element(By.ID, 's_swepi_1')
      input_password = driver.find_element(By.ID, 's_swepi_2')
      input_username.send_keys(self.username)
      input_password.send_keys(self.password)
      button_login = driver.find_element(By.ID, "s_swepi_22")
      button_login.click()
      self.attempts = 0
    except NoSuchElementException:
      if "The server you are trying to access is either busy or experiencing difficulties." in driver.find_element(By.TAG_NAME, "body").text:
        sleep(5)
        self.attempt(self.login)
      else:
        self.attempt(self.login)

  def open_leads_tab(self):
    print("opening leads tab")
    try:
      applet = driver.find_element(By.CLASS_NAME, "AppletTitle")
      if applet:
        print("applet")
        tabs = driver.find_elements(By.CLASS_NAME, "ui-corner-top")
        for tab in tabs:
          if tab.text == "Leads":
            tab.click()
            self.attempts = 0
            break
    except NoSuchElementException:
      self.attempt(self.open_leads_tab)

  def start_new_lead(self):
    try:
      button = driver.find_element(By.ID, "s_1_1_110_0_Ctrl")
      print("starting new lead")
      self.attempts = 0
      button.click()
    except NoSuchElementException:
      if self.attempts >= 50:
        self.attempts = 0
        raise Exception("attempt limit exceeded")
      print(f"attempting for the {self.attempts + 1} time")
      sleep(1)
      self.attempts += 1
      self.start_new_lead()

  def fill(self, customer_name, first_name, last_name, job_title, work_phone, email, address, city, state, zip_code):
    print("filling form")

    inputs = driver.find_elements(By.TAG_NAME, "input")

    for input in inputs:
      try:
        if input.get_attribute('name') == self.input_names['customer_name']:
          input.send_keys(customer_name)
        elif input.get_attribute('name') == self.input_names['first_name']:
          input.send_keys(first_name)
        elif input.get_attribute('name') == self.input_names['last_name']:
          input.send_keys(last_name)
        elif input.get_attribute('name') == self.input_names['job_title']:
          input.send_keys(job_title)
        elif input.get_attribute('name') == self.input_names['email'] and email:
          input.send_keys(email)
        elif input.get_attribute('name') == self.input_names['work_phone']:
          input.send_keys(work_phone)
        elif input.get_attribute('name') == self.input_names['address_line_1']:
          input.send_keys(address)
        elif input.get_attribute('name') == self.input_names['city']:
          input.send_keys(city)
        elif input.get_attribute('name') == self.input_names['state']:
          input.send_keys(state)
        elif input.get_attribute('name') == self.input_names['zip_code']:
          input.send_keys(zip_code)
        elif input.get_attribute('name') == self.input_names['acct_team_owner']:
          input.send_keys("JP.MacDonell")
        elif input.get_attribute('name') == self.input_names['franchise_name']:
          input.send_keys('Unishippers 1578')
        elif input.get_attribute('name') == self.input_names['primary_opportunity']:
          input.send_keys('US Express')
        elif input.get_attribute('name') == self.input_names['expected_quantity']:
          input.send_keys(100)
      except TypeError as e:
        print(e)
      except StaleElementReferenceException as e:
        print("stale element exception error")

  def accept_alert(self):
    try:
      alert = driver.switch_to.alert
      self.attempts = 0
      status = alert.text
      alert.accept()
      print("accepting alert")
      return status
    except NoAlertPresentException:
      return self.attempt(self.accept_alert)

  def check_loading(self):
    html = driver.find_element(By.TAG_NAME, "html")
    if html.get_attribute("class") == "siebui-busy":
      sleep(2.5)
      print("still saving")
      self.check_loading()
    else:
      return

  def save_form(self):
    print("saving form")
    action = ActionChains(driver)
    action.key_down(Keys.CONTROL).send_keys('S').key_up(Keys.CONTROL).perform()
    self.check_loading()

  def check_protection_status(self):
    sleep(.5)
    input = driver.find_element(By.CSS_SELECTOR, "input[name='s_1_1_34_0']")
    val = input.get_attribute("value")
    print(val)
    if val:
      self.attempts = 0
      print("checking protection status")
      return val
    else:
      return self.attempt(self.check_protection_status)

  def update_sales_stage(self):
    print("updating sales stage")
    input = driver.find_element(By.CSS_SELECTOR, "input[name='s_1_1_91_0']")
    input.clear()
    input.send_keys("Prospect")

  def request_green_light(self):
    print("requesting green light")
    driver.find_element(By.ID, 's_1_1_48_0_Ctrl').click()

  def determine_status(self):
    self.save_form()
    sleep(.5)
    protection_status = self.check_protection_status()
    sleep(.5)
    new_status = ""
    if "protected elsewhere" not in protection_status.lower():
      self.update_sales_stage()
      sleep(.5)
      self.save_form()
      sleep(.5)
      self.request_green_light()
      sleep(.5)
      new_status = self.accept_alert()
      self.attempts = 0
    else:
      new_status = protection_status
    print(new_status)
    return new_status

  def update_sheet(self, i, status):
    today = datetime.date.today()
    self.sheet.update(f"K{i+2}", status)
    self.sheet.update(f"L{i+2}", today.strftime("%B %d, %Y"))

  def close_popup(self, text="close"):
    for el in driver.find_elements(By.CLASS_NAME, "ui-button-text"):
      if el.text == text:
        el.click()

  def full_form(self, i, customer_name, first_name, last_name, job_title, work_phone, email, address, city, state, zip_code, status, updated):
    try:
      if (status):
        print(f"{i}", "\nstatus already exists:", status, updated)
      else:
        sleep(1)
        print(f"{i}", "\nno status, filling in fields")
        if len(state > 2):
            state = us_state_to_abbrev[state]
        self.fill(customer_name, first_name, last_name, job_title, work_phone, email, address, city, state, zip_code)
        status = self.determine_status()
        sleep(.5)
        self.update_sheet(i, status)
        sleep(.5)
        self.start_new_lead()
    except UnexpectedAlertPresentException as e:
      print(e)
      alert = driver.switch_to.alert
      self.update_sheet(i, f"UnexpectedAlertPresentException: {alert.text}. \n\nRow may have been skipped.")
      print("unexpected alert; skipping row")
      self.update_sheet(i, "Unexpected alert error; row skipped")
      self.accept_alert()
      self.attempts = 0
    except ElementClickInterceptedException as e:
      print(e)
      self.close_popup()
      sleep(.5)
      status = self.determine_status()
      self.update_sheet(i, status)
      sleep(.5)
      self.start_new_lead()
      try:
        self.request_green_light()
      except:
        self.update_sheet(i, "Element Click Intercepted Exception. Row likely skipped.")
    except KeyError as e:
      print(e)
      self.update_sheet(i, "Attribute Error. Unable to convert state name to ISO code. Row likely skipped.")

  def iterate(self):
    for i, customer_name, first_name, last_name, job_title, work_phone, email, address, city, state, zip_code, status, updated in unishippers.df.itertuples():
      print(datetime.datetime.now().strftime("%H:%M:%S"))
      self.full_form(i, customer_name, first_name, last_name, job_title, work_phone, email, address, city, state, zip_code, status, updated)
      print("\n\n")
      
unishippers = GreenLight(sheet)

In [14]:
unishippers.login()

attempting login for the 1 time
attempting login for the 2 time


In [52]:
unishippers.open_leads_tab()

opening leads tab
applet


In [53]:
unishippers.start_new_lead()

attempting for the 1 time
attempting for the 2 time
starting new lead


In [54]:
unishippers.iterate()

14:02:07
0 
status already exists: Value too long for field 'Address1' (maximum size 30).(SBL-DAT-00235)
 November 28, 2022



14:02:07
1 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) November 28, 2022



14:02:07
2 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) November 28, 2022



14:02:07
3 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) November 28, 2022



14:02:07
4 
status already exists: Protected Elsewhere November 28, 2022



14:02:07
5 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) November 28, 2022



14:02:07
6 
status already exists: Protected Elsewhere November 28, 2022



14:02:07
7 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) November 28, 2022



14:02:07
8 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) November 2

In [16]:
driver.quit()